IMPORT libraries and datasets

In [1]:
import pandas as pd
import numpy as np
import ast
import tensorflow as tf
from tensorflow.keras import layers, models

2024-09-23 13:34:16.755380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 13:34:16.772329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 13:34:16.776880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 13:34:16.789586: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.list_physical_devices('GPU')

I0000 00:00:1727098465.643944       8 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727098465.649055       8 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727098465.649090       8 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
dataset = pd.read_csv("/app/AI-Module/Resources/Datasets/how2sign.csv")

In [9]:
dataset.head()

,points,translation,id,len_keyframes
223,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...",We did some exercises today that are going to ...,0,427
242,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...","And now we're going to make a, it's called a, ...",1,282
358,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...",It should really spring back and feel dry. Thi...,2,335
492,"[[0.7839, 0.0, 1.0, 1.0, 0.5049, 0.1065, 0.702...",So let's begin with the basics. As you can see...,3,492
71,"[[0.8621, 0.0904, 1.0, 1.0, 0.5985, 0.0007, 0....",Hi! A lot of us has started it into the journe...,4,334


Modify datasets

In [5]:
import sys
# Agrega la ruta donde está ubicado Points2VecClass.py
sys.path.append('/app/AI-Module/Modules')
print(sys.version)
from Points2VecClass import Point2Vec
def pointsToCnnInputForm(video):
    p2v = Point2Vec(4)
    return p2v.CNNMatrix(video)

3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


Cambio el formato de points para que sea una lista de 48x48

In [4]:
dataset['points'] = dataset['points'].apply(ast.literal_eval)
dataset['points']=dataset['points'].apply(pointsToCnnInputForm)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fee53bec790>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fee53bec790>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [5]:
datset=dataset.sort_values(by='len_keyframes')

Tokenizar texto de traducciones para que sea compatible con el modelo


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Crear el tokenizador
tokenizer = Tokenizer()

# Ajustar el tokenizador al texto de las traducciones
tokenizer.fit_on_texts(df_train['translation'])

# Convertir las oraciones en secuencias de enteros
df_train['translation_sequence'] = tokenizer.texts_to_sequences(df_train['translation'])
df_val['translation_sequence'] = tokenizer.texts_to_sequences(df_val['translation'])
df_test['translation_sequence'] = tokenizer.texts_to_sequences(df_test['translation'])


Creo datasets de Tensorflow

In [ ]:
def create_tf_dataset(df):
    points = tf.ragged.constant(df['points'].values)
    sequence = tf.ragged.constant(df['translation_sequence'].values)
    return tf.data.Dataset.from_tensor_slices((points, sequence))

# train_dataset = create_tf_dataset(df_train)
# val_dataset = create_tf_dataset(df_val)
test_dataset = create_tf_dataset(df_test)

Hago el padding para que las secuencias dentro de los batches tengan la misma longitud

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_batch_fn(batch):
    points, sequences = batch
    
    # Encuentra la longitud máxima para los puntos
    points_lengths = tf.map_fn(lambda x: tf.shape(x)[0], points, dtype=tf.int32)
    max_len_points = tf.reduce_max(points_lengths)
    
    # Encuentra la longitud máxima para las secuencias
    sequences_lengths = tf.map_fn(lambda x: tf.shape(x)[0], sequences, dtype=tf.int32)
    max_len_sequences = tf.reduce_max(sequences_lengths)
    
    # Aplica el padding dinámico a los puntos usando numpy_function
    points_padded = tf.numpy_function(
        lambda pts: pad_sequences(pts, maxlen=max_len_points, dtype='float32', padding='post'), 
        [points], tf.float32
    )
    
    # Aplica el padding dinámico a las secuencias usando numpy_function
    sequences_padded = tf.numpy_function(
        lambda seqs: pad_sequences(seqs, maxlen=max_len_sequences, dtype='int32', padding='post'),
        [sequences], tf.int32
    )
    
    return points_padded, sequences_padded

# Función principal para mapear y agrupar en batches
def pad_batches(dataset):
    return dataset.map(pad_batch_fn).batch(32)

train_dataset = pad_batches(train_dataset)
val_dataset = pad_batches(val_dataset)
test_dataset = pad_batches(test_dataset)

**Buscar mejores parámetros**

In [ ]:
import keras_tuner as kt
num_classes=len(tokenizer.word_index) + 1


def ctc_loss(translation, prediction):
    
    input_length = tf.fill([tf.shape(prediction)[0], 1], tf.shape(prediction)[1])  # Longitud de la secuencia de entrada
    label_length = tf.math.count_nonzero(translation, axis=-1, keepdims=True)   # Longitud de la secuencia de traducción
    # Calcular la pérdida CTC usando ctc_batch_cost
    loss = tf.keras.backend.ctc_batch_cost(translation, prediction, input_length, label_length)
    return loss


# Definir el modelo para la búsqueda de hiperparámetros
def build_model(hp):

    def createCNNkt():

        model= models.Sequential()
        # Añadir capas CNN dinámicamente según el número de capas que se elija (1 a 5 capas CNN)
        for i in range(hp.Int('num_cnn_layers', 1, 5)):  # De 1 a 5 capas CNN
            model.add(layers.Conv2D(
                filters=hp.Int(f'conv_{i+1}_filters', min_value=32, max_value=128, step=32),
                kernel_size=(3, 3),
                activation='relu',
                input_shape=(48, 48, 1) if i == 0 else None))
            model.add(layers.MaxPooling2D((2, 2)))

        model.add(layers.Flatten())
        return model

    def createLSTMkt():

        model= models.Sequential()
        # Añadir capas LSTM dinámicamente según el número de capas que se elija (1 a 5 capas LSTM)
        for i in range(hp.Int('num_lstm_layers', 1, 5)):  # De 1 a 5 capas LSTM
            model.add(layers.LSTM(
                units=hp.Int(f'lstm_{i+1}_units', min_value=32, max_value=128, step=32),
                return_sequences=True))

            # Añadir Dropout para cada capa LSTM
            model.add(layers.Dropout(hp.Float(f'dropout_{i+1}', min_value=0.2, max_value=0.5, step=0.1)))
        return model

    cnn = createCNNkt()

    video_input = layers.Input(shape=(None, 48, 48, 1))

    # Aplicar CNN a cada frame usando TimeDistributed
    cnn_features = layers.TimeDistributed(cnn)(video_input)

    lstm=createLSTMkt()
    lstm_out= lstm(cnn_features)
    # Capa final de salida
    output = layers.Dense(num_classes, activation='linear')(lstm_out)

    # Compilar el modelo
    model.compile(optimizer='adam', loss=ctc_loss)

    model = models.Model(inputs=video_input, outputs=output)

    return model

# Definir la búsqueda de hiperparámetros con Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_loss', # Métrica de evaluación
    max_epochs= 50,
    factor=3,
    directory='',
    project_name='cnn_lstm_hyperparam'
)

In [ ]:
# Realizar la búsqueda de hiperparámetros
tuner.search(train_dataset, validation_data=val_dataset)

# Obtener los mejores hiperparámetros encontrados
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0].values()

Crear modelo,**EJEMPLO**

CNN

In [ ]:
def create_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    return model

LSTM

In [ ]:
def create_lstm():
    model=models.Sequential()
    # Primera capa LSTM con return_sequences=True
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dropout(0.3)) # Añadir Dropout
    
    # Segunda capa LSTM con return_sequences=True
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dropout(0.3))

    # Tercera capa LSTM con return_sequences=False
    model.add(layers.LSTM(64, return_sequences=False))
    model.add(layers.Dropout(0.2))
    return model

CNN+LSTM

In [ ]:
def create_cnn_lstm_model(num_classes):
    cnn = create_cnn()
    
    video_input = layers.Input(shape=(None, 48, 48, 1)) 
    
    # Aplicar CNN a cada frame usando TimeDistributed
    cnn_features = layers.TimeDistributed(cnn)(video_input)
    
    lstm=create_lstm()
    lstm_out= lstm(cnn_features)
    # Capa final de salida
    output = layers.Dense(num_classes, activation='softmax')(lstm_out)
    
    model = models.Model(inputs=video_input, outputs=output)
    return model

Implementación para el entrenamiento

Usar bleu score

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(references, hypotheses):
    smoothing = SmoothingFunction().method4
    return np.mean([sentence_bleu([ref], hyp, smoothing_function=smoothing) for ref, hyp in zip(references, hypotheses)])

In [ ]:
model = create_cnn_lstm_model(num_classes)

# Compilar el modelo
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
optimizador = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizador)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='RUTA',  # Ruta donde se guarda el modelo
    save_best_only=True,       # Guardar solo si es el mejor modelo hasta ahora
    monitor='val_bleu',        # Métrica a monitorear
    mode='max',
    monitor= 'val_loss',
    mode='min'                 # Modo: minimizar la métrica monitorizada
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
loss_metric=tf.keras.backend.ctc_batch_cost(from_logits=True, reduction= None)

In [ ]:
def trainstep(model, points_batch, translation_batch,step, epoch_loss_avg, epoch_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(points_batch, training=True)
        loss = tf.reduce_mean(loss_metric(translation_batch, predictions))
        
    # Calcular y aplicar gradientes
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if step % 50 == 0:
        print(f"Step {step}: loss = {loss.numpy()}")
    epoch_accuracy.update_state(translation_batch, predictions)
    epoch_loss_avg.update_state(loss)

In [ ]:
def valstep(model, points_batch, translation_batch):
    val_loss, val_acc = model.evaluate(x=points_batch, y=translation_batch)
    print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")
    
    # Evaluar en el conjunto de validación al final de cada epoch usando BLEU
    val_references = [tokenizer.sequences_to_texts([ref.numpy()]) for ref in translation_batch]
    val_predictions = model.predict(points_batch)
    val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in np.argmax(val_predictions, axis=-1)]
    val_bleu = calculate_bleu(references = val_references, hypotheses = val_hypotheses)
    print(f"Validation BLEU: {val_bleu}")
    return val_acc,val_loss,val_bleu

In [ ]:
# Entrenar el modelo usando un ciclo de entrenamiento personalizado
epochs = 100

# Iterar a través de los batches y ajustar manualmente los parámetros
for epoch in range(epochs):
    
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    print(f"Epoch {epoch + 1}/{epochs}")
    
    for step, (points_batch, translation_batch) in enumerate(train_dataset):
        trainstep(model, points_batch,translation_batch, step, epoch_loss_avg, epoch_accuracy)
    
    
    for step, (points_batch,translation_batch) in enumerate(val_dataset):
        val_acc, val_loss, val_bleu = valstep(model, points_batch, translation_batch)

    checkpoint_callback.on_epoch_end(epoch, logs={'val_loss': val_loss, 'val_accuracy': val_acc, 'val_bleu': val_bleu})

    if early_stopping.on_epoch_end(epoch, logs={'val_loss': val_loss, 'val_accuracy': val_acc, 'val_bleu': val_bleu}):
        print("Early stopping triggered")
        break
    print(f"Epoch: {epoch}, Loss: {epoch_loss_avg}, Accuracy: {epoch_accuracy}")

# Evaluar el modelo en el dataset de test
#test_loss, test_acc = model.evaluate(test_dataset)
#print(f'Test accuracy: {test_acc}')